In [1]:
# !pip install opencv-python
# !pip install opencv-contrib-python

In [1]:
import cv2
from cv2 import dnn_superres
import moviepy
import moviepy.editor
import os

file_name = 'low_res_2' # 'Megan Is Missing'
ext = 'mp4'
scale = 4
input_video = f'./input/videos/{file_name}.{ext}'
output_root_path = f'./output/videos/{file_name}_multiproc'

os.makedirs(output_root_path, exist_ok=True)


In [2]:
import torch
import os
import ray
import sys
sys.path.append(os.path.abspath('./input/BSRGAN'))
from models.network_rrdbnet import RRDBNet as net

threads = 3
runtime_env = {"working_dir": "./", "conda": "env_pytorch"}
num_cpus = threads
num_gpus = 1/threads
ray.init(runtime_env=runtime_env, num_cpus=4, num_gpus=1)

model_name = 'BSRGAN'
modelScale = 4
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_path = f'./input/models/{model_name}.pth'
time_limit = 1*60 # seconds

torch.cuda.empty_cache()

model = net(in_nc=3, out_nc=3, nf=64, nb=23, gc=32, sf=modelScale)  # define network
model.load_state_dict(torch.load(model_path), strict=True)
model = model.to(device)
modelObjRef = ray.put(model)
modelObjRef

2023-06-03 12:07:22,338	INFO worker.py:1625 -- Started a local Ray instance.
2023-06-03 12:07:22,362	INFO packaging.py:520 -- Creating a file package for local directory './'.
2023-06-03 12:07:22,384	INFO packaging.py:347 -- Pushing file package 'gcs://_ray_pkg_b6bdab388c25f83b.zip' (0.09MiB) to Ray cluster...
2023-06-03 12:07:22,387	INFO packaging.py:360 -- Successfully pushed file package 'gcs://_ray_pkg_b6bdab388c25f83b.zip'.


[3, 3, 64, 23, 32, 4]


ObjectRef(00ffffffffffffffffffffffffffffffffffffff0100000001000000)

In [3]:
def get_output_file_path(file_name, model_name, file_part, parent_path, ext):
    file_prefix = f'{file_name}_{model_name}_{file_part}'
    output_path = f'{parent_path}/{file_prefix}.{ext}'
    return output_path

### Split input video into #threads number of parts

In [ ]:
vcap = cv2.VideoCapture(input_video)
input_video_parts_root = f'./input/videos/{file_name}_parts'
os.makedirs(input_video_parts_root, exist_ok=True)

total_frames = vcap.get(cv2.CAP_PROP_FRAME_COUNT)
frames_per_part = total_frames//threads
codec = cv2.VideoWriter_fourcc(*'mp4v')
frame_rate = vcap.get(cv2.CAP_PROP_FPS)
success, frame = vcap.read()
frame_count = 0
vcap.set(cv2.CAP_PROP_POS_FRAMES, 0)
input_width = frame.shape[1]
input_height = frame.shape[0]
out_resolution = (input_width, input_height)

In [4]:
for part in range(threads):
    part_path = f'{input_video_parts_root}/{file_name}_part{part}.{ext}'
    print(f'writing to {part_path}')
    vwriter = cv2.VideoWriter(part_path, codec,
                         frame_rate, out_resolution)
    start = int(frames_per_part * part)
    end = int(start + frames_per_part if part < threads-1 else total_frames)
    for i in range(start, end):
        success, frame = vcap.read()
        vwriter.write(frame)
    
    vwriter.release()

vcap.release()

file_parts = os.listdir(input_video_parts_root)
num_frames = 0
for file_part_ in file_parts:
    vs = cv2.VideoCapture(f'{input_video_parts_root}/{file_part_}')
    num_frames += vs.get(cv2.CAP_PROP_FRAME_COUNT)

print(f'total frames in input={total_frames}, total frames in parts={num_frames}')

writing to ./input/videos/low_res_2_parts/low_res_2_part0.mp4
writing to ./input/videos/low_res_2_parts/low_res_2_part1.mp4
writing to ./input/videos/low_res_2_parts/low_res_2_part2.mp4
total frames in input=913.0, total frames in parts=913.0


In [5]:
from tqdm.auto import tqdm

@ray.remote(num_gpus=num_gpus, num_cpus=num_cpus)
class WorkerActor(object):
    def __init__(self, model, video_path, output_path) -> None:
        self.model = model #.to('cuda')
        self.input_path = video_path
        self.output_path = output_path

    def upscale(self):
        codec = cv2.VideoWriter_fourcc(*'mp4v')
        output_part_path = self.output_path  # get_output_file_path(file_name, model_name, f'part{part}', output_root_path, ext)
        
        input_part_path = self.input_path  # f'{input_video_parts_root}/{file_name}_part{part}.{ext}'
        vs = cv2.VideoCapture(input_part_path)
        total_frames = vs.get(cv2.CAP_PROP_FRAME_COUNT)
        print(vs, input_part_path, total_frames)
        # vs.set(cv2.CAP_PROP_POS_FRAMES, number_of_frames_completed)
        success, frame = vs.read()
        input_width = frame.shape[1] # 640
        input_height = frame.shape[0] # 352
        frame_count = 0 # number_of_frames_completed
        frame_rate = vs.get(cv2.CAP_PROP_FPS)
        out_resolution = (modelScale*input_width, modelScale*input_height)

        out = cv2.VideoWriter(output_part_path, codec,
                            frame_rate, out_resolution)

        print(f'input res: {frame.shape}, total frames={total_frames},\
            frame rate={frame_rate}, scale={modelScale},\
            outpath={output_part_path}')

        # with prof(period=0.001):
        with tqdm(total=total_frames) as pbar:
            # pbar.update(number_of_frames_completed)

            while success:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = util.uint2tensor4(frame).to(device)
                upsampled_frame = model(frame)
                upsampled_frame = util.tensor2uint(upsampled_frame)
                upsampled_frame = cv2.cvtColor(upsampled_frame, cv2.COLOR_RGB2BGR)
                
                out.write(upsampled_frame)
                frame_count += 1
                pbar.update(1)
                pbar.set_description(f'frame no: {frame_count}')
                success, frame = vs.read()
                # if frame_count % 50:
                #     # check if run time > time_limit
                #     if time.time() - start_time > time_limit:
                #         break
                    
            pbar.close()
        # prof.print_stats()

        vs.release()
        out.release()
        print(f'finished part={part}, {frame_count} frames ({frame_count/total_frames*100}%)')
    
    def get_id(self):
        return id(self.model)

In [6]:
workers = []
for i in range(threads):
    output_path = get_output_file_path(file_name, model_name, f'part{i}', output_root_path, ext)
    input_path = f'{input_video_parts_root}/{file_name}_part{i}.{ext}'
    workers.append(WorkerActor.remote(modelObjRef, input_path, output_path))
    print(f'{workers[-1]}-->{input_path} to {output_path}')

2023-06-03 12:08:06,082	WARNING worker.py:1986 -- Warning: The actor WorkerActor is very large (64 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.


Actor(WorkerActor, 2ece7db0e5e6e1af6edb3f2401000000)-->./input/videos/low_res_2_parts/low_res_2_part0.mp4 to ./output/videos/low_res_2_multiproc/low_res_2_BSRGAN_part0.mp4
Actor(WorkerActor, 393664afe86cc6ec013e4be601000000)-->./input/videos/low_res_2_parts/low_res_2_part1.mp4 to ./output/videos/low_res_2_multiproc/low_res_2_BSRGAN_part1.mp4
Actor(WorkerActor, 9743942dacfd13cdc82c5e1401000000)-->./input/videos/low_res_2_parts/low_res_2_part2.mp4 to ./output/videos/low_res_2_multiproc/low_res_2_BSRGAN_part2.mp4


(TemporaryActor pid=10647) 2023-06-03 12:08:09,570	ERROR serialization.py:387 -- No module named 'models'
(TemporaryActor pid=10647) Traceback (most recent call last):
(TemporaryActor pid=10647)   File "/home/asutosh/miniconda3/envs/env_pytorch/lib/python3.8/site-packages/ray/_private/serialization.py", line 385, in deserialize_objects
(TemporaryActor pid=10647)     obj = self._deserialize_object(data, metadata, object_ref)
(TemporaryActor pid=10647)   File "/home/asutosh/miniconda3/envs/env_pytorch/lib/python3.8/site-packages/ray/_private/serialization.py", line 268, in _deserialize_object
(TemporaryActor pid=10647)     return self._deserialize_msgpack_data(data, metadata_fields)
(TemporaryActor pid=10647)   File "/home/asutosh/miniconda3/envs/env_pytorch/lib/python3.8/site-packages/ray/_private/serialization.py", line 223, in _deserialize_msgpack_data
(TemporaryActor pid=10647)     python_objects = self._deserialize_pickle5_data(pickle5_data)
(TemporaryActor pid=10647)   File "/home/

### Upscale video multiprocessing

In [7]:
for worker in workers:
    worker.upscale.remote()

In [ ]:
from pathlib import Path

def concatenate_videos(new_video_path, codec, fps, resolution, videos):
    video = cv2.VideoWriter(new_video_path, codec, fps, resolution)

    for v in tqdm(videos):
        curr_v = cv2.VideoCapture(v)
        while curr_v.isOpened():
            r, frame = curr_v.read()
            if not r:
                break
            video.write(frame)

    video.release()
    print(f'video combined at {new_video_path}')

files_sorted = list(map(str, sorted(Path(output_root_path).iterdir(), key=os.path.basename)))
concat_path = get_output_file_path(file_name, model_name, 'all', output_root_path, ext)
concatenate_videos(concat_path, codec, frame_rate, out_resolution, files_sorted)

In [ ]:
from moviepy.editor import AudioFileClip

video = moviepy.editor.VideoFileClip(input_video)
audio = video.audio
audio

In [ ]:
video_out = moviepy.editor.VideoFileClip(concat_path)
audio = audio.volumex(2)
video_with_audio = video_out.set_audio(audio)
output_video_audio = get_output_file_path(file_name, model_name, 'final', output_root_path, ext)
video_with_audio.write_videofile(output_video_audio)